In [1]:
import gym
import numpy as np
from collections import defaultdict
from collections import deque
import sys
import math
import numpy as np
from IPython.display import clear_output
from time import sleep

In [2]:
class Agent:

    def __init__(self, nA=6, alpha = 0.1, gamma = 0.9):
        """ Initialize agent.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        #self.epsilon = 0.9
        self.alpha = alpha
        self.i_episode = 0
        self.gamma = gamma

    def update_Q(self, Qsa, Qsa_next, reward, alpha, gamma):
    # updates the action-value function estimate using the most recent time step 
        return Qsa + (alpha * (reward + (gamma * Qsa_next) - Qsa))

    def epsilon_greedy_probs(self, Q_s, i_episode, eps=None):
        """ obtains the action probabilities corresponding to epsilon-greedy policy """
        #epsilon = 1.0 / i_episode
        #testing other decay rates
        if i_episode<=12000:
            epsilon = -(0.9/12000)*i_episode+1
        else:
            epsilon = 0.001
        if eps is not None:
            epsilon = eps
        policy_s = np.ones(self.nA) * epsilon / self.nA
        policy_s[np.argmax(Q_s)] = 1 - epsilon + (epsilon / self.nA)
        return policy_s

    def select_action(self, state, debug = False):
        """ Given the state, select an action.

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        self.i_episode += 1
        policy_s = self.epsilon_greedy_probs(self.Q[state], self.i_episode)
        action = np.random.choice(self.nA, p=policy_s)
        return action

    def step(self, state, action, reward, next_state, done):
        """ Update the agent's knowledge, using the most recently sampled tuple.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        self.Q[state][action] = self.update_Q(self.Q[state][action], np.max(self.Q[next_state]), reward, self.alpha, self.gamma)
    
    def select_greedy_action(self, state):
        #policy_sarsamax = np.array([np.argmax(self.Q[key]) if key in self.Q else -1 for key in np.arange(500)])
        return np.argmax(self.Q[state])

In [3]:
def interact(env, agent, num_episodes=20000, window=100):
    """ Monitor agent's performance.
    
    Params
    ======
    - env: instance of OpenAI Gym's Taxi-v1 environment
    - agent: instance of class Agent (see Agent.py for details)
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)
    # for each episode
    for i_episode in range(1, num_episodes+1):
        # begin the episode
        state = env.reset()
        # initialize the sampled reward
        samp_reward = 0
        #frames = []
        while True:
            # agent selects an action
            action = agent.select_action(state)
            # agent performs the selected action
            #env.render()
            next_state, reward, done, _ = env.step(action)
            '''
            frames.append({
                    'frame': env.render(mode='ansi'),
                    'state': state,
                    'action': action,
                    'reward': reward
                    }
                )'''
            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done)
            # update the sampled reward
            samp_reward += reward
            # update the state (s <- s') to next time step
            state = next_state
            
            if done:
                # save final sampled reward
                samp_rewards.append(samp_reward)
                #print_frames(frames)
                break
        if (i_episode >= 100):
            # get average reward from last 100 episodes
            avg_reward = np.mean(samp_rewards)
            # append to deque
            avg_rewards.append(avg_reward)
            # update best average reward
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
        # monitor progress
        print("\rEpisode {}/{} || Best average reward {}".format(i_episode, num_episodes, best_avg_reward), end="")
        sys.stdout.flush()
        # check if task is solved (according to OpenAI Gym)
        if best_avg_reward >= 9.7:
            print('\nEnvironment solved in {} episodes.'.format(i_episode), end="")
            break
        if i_episode == num_episodes: print('\n')
    return avg_rewards, best_avg_reward

In [4]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.7)

In [5]:
def run_estimated_policy(agent, env):
    epochs = 0
    penalties, reward = 0, 0
    frames = []
    done = False
    state = env.reset()

    while not done:
        action = agent.select_greedy_action(state)
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        # Put each rendered frame into dict for animation
        frames.append({
                'frame': env.render(mode='ansi'),
                'state': state,
                'action': action,
                'reward': reward
                    }
                )

        epochs += 1

    print_frames(frames)
    print("Timesteps taken: {}".format(epochs))
    print("Penalties incurred: {}".format(penalties))

In [6]:
env = gym.make('Taxi-v2')
#initializing the agent
agent = Agent(alpha = 0.1, gamma = 0.9)
#interacts with the environtment during num_episodes
avg_rewards, best_avg_reward = interact(env, agent, num_episodes = 20000)

Episode 20000/20000 || Best average reward 9.117



In [11]:
#shows the estimated optimal policy rendering the environment
run_estimated_policy(agent, env)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 14
State: 410
Action: 5
Reward: 20
Timesteps taken: 14
Penalties incurred: 0
